In [1]:
from __future__ import print_function

import warnings

warnings.filterwarnings('ignore')

import argparse
import torch
import torch.utils.data
from torch.utils.data import DataLoader, TensorDataset
from torch import nn, optim
from torch.nn import functional as F

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import get_data, utils

In [2]:
# Set up experiment parameters
params = utils.Params("model_configurations/experiment_params.json")
X, y, cols = get_data.get_and_preprocess_german(params)

features = [c for c in X]

X = X.values
scaler = MinMaxScaler().fit(X)
X = scaler.transform(X)
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.1)

In [3]:
device = torch.device("cuda")

kwargs = {'num_workers': 1, 'pin_memory': True}

# Prepare data loader
xtrain = torch.from_numpy(xtrain)
ytrain = torch.from_numpy(ytrain)
xtest = torch.from_numpy(xtest)
ytest = torch.from_numpy(ytest)

train_dataset = TensorDataset(xtrain, ytrain)
test_dataset = TensorDataset(xtest, ytest)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, **kwargs)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True, **kwargs)

In [4]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        # TODO neural net hyper-parameters
        """self.fc1 = nn.Linear(28, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 28)"""

        self.fc1 = nn.Linear(28, 60)
        self.fc21 = nn.Linear(60, 30)
        self.fc22 = nn.Linear(60, 30)
        self.fc3 = nn.Linear(30, 60)
        self.fc4 = nn.Linear(60, 28)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 28))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [5]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [6]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [7]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.float().to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.float().to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            """
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)
            """
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


In [22]:
def main():
    for epoch in range(1, 30 + 1):
        train(epoch)
        test(epoch)

        if 1:
            torch.save(model.state_dict(), "vae_lime_testing.pt")



    with torch.no_grad():
        sample = torch.randn(5, 30).to(device)
        sample = model.decode(sample).cpu()

        # TODO Inverse transform not one-hot ?!
        inversed = scaler.inverse_transform(sample)
        np.set_printoptions(suppress=True)
        #print(sample)
        #print(inversed)

        s = [np.round(i, 0) for i in inversed]
        for a in s:
            print(a)


In [12]:
# no training
main()

====> Test set loss: 19.7705
====> Test set loss: 19.8021
====> Test set loss: 19.9034
====> Test set loss: 19.8714
====> Test set loss: 19.7822
====> Test set loss: 19.7792
====> Test set loss: 19.8129
====> Test set loss: 19.8766
====> Test set loss: 19.8747
====> Test set loss: 19.7833
====> Test set loss: 19.9988
====> Test set loss: 19.8235
====> Test set loss: 19.8172
====> Test set loss: 19.8282
====> Test set loss: 19.9171
====> Test set loss: 19.7358
====> Test set loss: 19.8315
====> Test set loss: 19.8512
====> Test set loss: 19.7672
====> Test set loss: 19.8054
====> Test set loss: 19.8623
====> Test set loss: 19.8973
====> Test set loss: 19.8089
====> Test set loss: 19.7947
====> Test set loss: 19.8538
====> Test set loss: 19.8677
====> Test set loss: 19.7702
====> Test set loss: 19.8000
====> Test set loss: 19.8709
====> Test set loss: 19.7920
[   0.    0.    0.   48.   32. 9323.    2.    2.    2.    2.    1.    1.
    1.    0.    0.    0.    1.    1.    0.    0.    1.   

In [14]:
main()

Train Epoch: 1 [0/900 (0%)]	Loss: 19.698208
====> Epoch: 1 Average loss: 19.2039
====> Test set loss: 18.4050
Train Epoch: 2 [0/900 (0%)]	Loss: 18.346918
====> Epoch: 2 Average loss: 18.0209
====> Test set loss: 17.4065
Train Epoch: 3 [0/900 (0%)]	Loss: 17.498837
====> Epoch: 3 Average loss: 17.1053
====> Test set loss: 16.7458
Train Epoch: 4 [0/900 (0%)]	Loss: 16.747719
====> Epoch: 4 Average loss: 16.2959
====> Test set loss: 15.8448
Train Epoch: 5 [0/900 (0%)]	Loss: 16.025314
====> Epoch: 5 Average loss: 15.6411
====> Test set loss: 15.2137
Train Epoch: 6 [0/900 (0%)]	Loss: 15.101856
====> Epoch: 6 Average loss: 14.9783
====> Test set loss: 14.3833
Train Epoch: 7 [0/900 (0%)]	Loss: 14.597377
====> Epoch: 7 Average loss: 14.5265
====> Test set loss: 14.0580
Train Epoch: 8 [0/900 (0%)]	Loss: 14.353107
====> Epoch: 8 Average loss: 14.0996
====> Test set loss: 13.7541
Train Epoch: 9 [0/900 (0%)]	Loss: 13.896695
====> Epoch: 9 Average loss: 13.9008
====> Test set loss: 13.4982
Train Epoc

In [24]:
model = VAE().to(device)
model.load_state_dict(torch.load("vae_lime.pt"))
model.eval()

VAE(
  (fc1): Linear(in_features=28, out_features=60, bias=True)
  (fc21): Linear(in_features=60, out_features=30, bias=True)
  (fc22): Linear(in_features=60, out_features=30, bias=True)
  (fc3): Linear(in_features=30, out_features=60, bias=True)
  (fc4): Linear(in_features=60, out_features=28, bias=True)
)